# SETUP

In [ ]:
!nvidia-smi

## Install Automatic1111's Gradio Web UI

In [ ]:
!git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

In [ ]:
%pip install -r requirements.txt
!mkdir repositories
!git clone --depth 1 https://github.com/CompVis/stable-diffusion.git repositories/stable-diffusion
!git clone --depth 1 https://github.com/CompVis/taming-transformers.git repositories/taming-transformers
!git clone --depth 1 https://github.com/CompVis/latent-diffusion.git repositories/latent-diffusion
!git clone --depth 1 https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
!git clone --depth 1 https://github.com/crowsonkb/k-diffusion.git repositories/k-diffusion
!git clone --depth 1 https://github.com/salesforce/BLIP.git repositories/BLIP
%pip install -r repositories/CodeFormer/requirements.txt

## Download SD v1.4 model from Huggingface
You'll need a token from HuggingFace for this, and will need to agree to the T&C of the repo.

Alternatively, upload it to your drive / storage and copy it over.

In [ ]:
!mkdir -p models/Stable-diffusion
!wget -c --header="Authorization: Bearer Your_Token_Here" "https://huggingface.co/CompVis/username/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O models/Stable-diffusion/model.ckpt"

## GFPGAN

In [ ]:
!wget -c https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth

### Use GPU to load models

In [ ]:
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

def replace_line_in_file(file_name, line_to_search, text_to_replace):
    with open(file_name, 'r') as file:
        # read a list of lines into data
        data = file.readlines()

    for line in data:
        # if the line contains the string we're looking for,
        # write the line to the output file
        if line_to_search in line:
            replace_line(file_name, data.index(line), text_to_replace)

replace_line_in_file('modules/sd_models.py', 'pl_sd = torch.load(checkpoint_file, map_location="cpu")', '    pl_sd = torch.load(checkpoint_file, map_location="cuda:0")\n')

### Change maximum steps to 500
Optional.  I personally find 150 to be very limiting.

Also augments img2img steps to 500.

In [ ]:
!wget http://0x0.st/oWuq.json -O ui-config.json

### Downloading Models from HF with wget
Replace 'username' and 'dataset' with whatever you wish to download.

In [ ]:
#!wget -c --header="Authorization: Bearer Your_Token_Here" "https://huggingface.co/datasets/username/dataset/resolve/main/dataset.ckpt" -O models/Stable-diffusion/dataset.ckpt

### Download concurrently
To save time.

In [ ]:
#!apt -y install parallel

Use `--quiet` to get rid of an avalanche of download output from wget once the `parallel` job finishes.

In [ ]:
#!echo '!wget -c --quiet --header="Authorization: Bearer Your_Token_Here" "https://huggingface.co/datasets/username/dataset/resolve/main/dataset.ckpt" -O models/Stable-diffusion/mymodel.ckpt' > dllist
#!echo '!wget -c --quiet --header="Authorization: Bearer Your_Token_Here" "https://huggingface.co/datasets/username/dataset/resolve2/main/dataset.ckpt" -O models/Stable-diffusion/mymodel2.ckpt' >> dllist
#!cat dllist

In [ ]:
#cat dllist | parallel 

## Install Xformers

If installing xformers takes too long, use the provided wheel (`.whl` file) here.  Will work for a T4 GPU.

Alternatively, save your `whl` files after the first run, to save time on the next instance.  Probably does not work cross-GPU.

See https://old.reddit.com/r/StableDiffusion/comments/xphaiw/dreambooth_stable_diffusion_training_in_just_125/iq66lw0/ for details.

In [ ]:
%pip install -U --pre triton
%pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers
# %pip install https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# Use the above for a much faster install

### Overwrite files to make use of xformers

In [ ]:
!wget 'https://pastebin.com/raw/BEdeV3wG' -O repositories/stable-diffusion/ldm/modules/attention.py


# Launch web ui.


https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Optimizations

https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Run-with-Custom-Parameters

In [ ]:
%cd /content/stable-diffusion-webui
!python webui.py --share

# Export your work

You'll need to install `magic-wormhole` on your PC to receive files.  Run `pip install magic-wormhole`

In [ ]:
#%pip install magic-wormhole
#!tar cf out.tar outputs
#!wormhole send out.tar

# Notes

## Sources Used:

Modified from https://github.com/daswer123/stable-diffusion-colab/blob/main/StableDiffusionUI_Voldemort.ipynb

xformers added from https://github.com/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb and https://github.com/ShivamShrirao/diffusers/blob/main/examples/dreambooth/train_dreambooth.py

## Saving outputs

You can use an S3 bucket on platforms that allow you to access `fusermount`.  Installing `s3fs`:  `apt install s3fs`

Alternatively `tar` your output folder and send it over with `croc` or `magic-wormhole`.